#### Capstone Project Appendix
#### Viewing Seattle Collisions in GeoHeatMap (2016 - 2019)

##### (If this is being viewed in __GitHub regular mode__ and the map does not render, it is recommended to switch to **nbviewer mode** by clicking the button below the trash bin iconso the map will render properly.)

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import folium
from branca.element import Figure
from folium.plugins import HeatMapWithTime
from folium import plugins
from IPython.core.display import display, HTML
!pip install folium --upgrade

Requirement already up-to-date: folium in /opt/anaconda3/lib/python3.8/site-packages (0.11.0)


In [2]:
#preparing the data
geo = pd.read_csv('sdot_2016_2019.csv')
geo['INCDATE'] = pd.to_datetime(geo['INCDATE'])
geo['INCDTTM'] = pd.to_datetime(geo['INCDTTM'])
geo['hour'] = geo['INCDTTM'].dt.hour
pd.options.display.max_columns = None
display(geo)

,Unnamed: 0,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,year,dayofweek,week_number,month,weekend,LocNaN,hour
0,0,-122.386772,47.564720,1,326234,327734,E984735,Matched,Intersection,31893.0,CALIFORNIA AVE SW AND SW GENESEE ST,,NaN,2,Injury Collision,Pedestrian,2,1,0,1,1,0,0,2019-11-20 00:00:00+00:00,2019-11-20 10:25:00,At Intersection (intersection related),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,N,Clear,Dry,Daylight,Y,NaN,NaN,2,Vehicle turning left hits pedestrian,0,0,N,2019,2,47,11,0,False,10
1,1,-122.341806,47.686934,2,326246,327746,E985430,Matched,Intersection,24228.0,STONE AVE N AND N 80TH ST,,NaN,2,Injury Collision,Angles,4,0,0,2,2,0,0,2019-11-18 00:00:00+00:00,2019-11-18 17:01:00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Raining,Wet,Dark - Street Lights On,NaN,NaN,NaN,10,Entering at angle,0,0,N,2019,0,47,11,0,False,17
2,6,-122.323569,47.722577,7,320602,322102,3796466,Matched,Block,NaN,5TH AVE NE BETWEEN 130TH ST OFF RP AND NE 130T...,,NaN,1,Property Damage Only Collision,Other,2,0,0,2,0,0,0,2019-06-16 00:00:00+00:00,2019-06-16 01:07:00,Mid-Block (not related to intersection),28.0,MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT,NaN,N,Clear,Dry,Dark - Street Lights On,NaN,NaN,NaN,50,Fixed object,0,0,N,2019,6,24,6,1,False,1
3,8,-122.328878,47.733182,9,326166,327666,3751600,Matched,Block,NaN,1ST AVE NE BETWEEN N 143RD ST AND NE 145TH ST,,NaN,2,Injury Collision,Pedestrian,2,1,0,1,1,0,0,2019-11-21 00:00:00+00:00,2019-11-21 18:08:00,Mid-Block (not related to intersection),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,N,Clear,Dry,Dark - Street Lights On,Y,NaN,NaN,0,Vehicle going straight hits pedestrian,0,521184,N,2019,3,47,11,0,False,18
4,12,-122.332365,47.665028,13,320518,322018,3646216,Matched,Block,NaN,N 50TH ST BETWEEN MERIDIAN AVE N AND KEYSTONE ...,,NaN,1,Property Damage Only Collision,Rear Ended,4,0,0,2,0,0,0,2019-06-21 00:00:00+00:00,2019-06-21 21:45:00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Clear,Dry,Dark - Street Lights On,NaN,NaN,NaN,14,From same direction - both going straight - on...,0,0,N,2019,4,25,6,1,False,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49829,221261,-122.329060,47.605233,221262,318579,320079,E909186,Matched,Block,NaN,6TH AVE BETWEEN CHERRY ST AND COLUMBIA ST,,NaN,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2019-03-07 00:00:00+00:00,2019-03-07 14:00:00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Unknown,Unknown,Daylight,NaN,NaN,NaN,14,From same direction - both going straight - on...,0,0,N,2019,3,10,3,0,False,14
49830,221262,-122.314961,47.523468,221263,318368,319868,3780990,Matched,Block,NaN,14TH AVE S BETWEEN S CONCORD ST AND S HENDERSO...,,NaN,2,Injury Collision,Rear Ended,2,0,0,2,1,0,0,2019-04-26 00:00:00+00:00,2019-04-26 09:59:00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,14,From same direction - both going straight - on...,0,0,N,2019,4,17,4,1,False,9
49831,221263,-122.334203,47.575576,221264,318948,320448,E920056,Matched,Intersection,31379.0,1ST AVE S AND S HANFORD ST,,NaN,1,Property Damage Only Collision,Right Turn,2,0,0,2,0,0,0,2019-05-09 00:00:00+00:00,2019-05-09 07:00:00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,16,From sa

In [3]:
sdot = pd.read_csv('sdot_2016_2019.csv')
sdot['LocNaN'] = pd.Series(dtype='object')
sdot['LocNaN'] = np.where(sdot.Y.isna(), 'True', 'False')
geo_s = geo[geo.LocNaN !=True]  #Removing rows with Y or X NaN


In [4]:
geo_s.shape[0]   #Confirming that the X or Y NaN rows have been removed 


48292

In [5]:
#drop unneeded columns
geo_s = geo_s[['X','Y','INCKEY','SEVERITYDESC','INCDTTM','hour','weekend','WEATHER','ROADCOND','LIGHTCOND','ADDRTYPE','JUNCTIONTYPE']]
geo_s.head()


,X,Y,INCKEY,SEVERITYDESC,INCDTTM,hour,weekend,WEATHER,ROADCOND,LIGHTCOND,ADDRTYPE,JUNCTIONTYPE
0,-122.386772,47.564720,326234,Injury Collision,2019-11-20 10:25:00,10,0,Clear,Dry,Daylight,Intersection,At Intersection (intersection related)
1,-122.341806,47.686934,326246,Injury Collision,2019-11-18 17:01:00,17,0,Raining,Wet,Dark - Street Lights On,Intersection,At Intersection (intersection related)
2,-122.323569,47.722577,320602,Property Damage Only Collision,2019-06-16 01:07:00,1,1,Clear,Dry,Dark - Street Lights On,Block,Mid-Block (not related to intersection)
3,-122.328878,47.733182,326166,Injury Collision,2019-11-21 18:08:00,18,0,Clear,Dry,Dark - Street Lights On,Block,Mid-Block (not related to intersection)
4,-122.332365,47.665028,320518,Property Damage Only Collision,2019-06-21 21:45:00,21,1,Clear,Dry,Dark - Street Lights On,Block,Mid-Block (not related to intersection)


In [6]:
geo_s.duplicated().value_counts()  #checking if duplicates
geo_s=geo_s.drop_duplicates()      #removing duplicates
geo_s.isnull().sum()               #checking if any null


X                  0
Y                  0
INCKEY             0
SEVERITYDESC       0
INCDTTM            0
hour               0
weekend            0
WEATHER         8381
ROADCOND        8332
LIGHTCOND       8408
ADDRTYPE           0
JUNCTIONTYPE    2680
dtype: int64

In [7]:
#We'll look at weekdays and weekends separately

geo_s_WE = geo_s[geo_s.weekend == 1]  #Weekends
geo_s_WD = geo_s[geo_s.weekend == 0]  #Weekdays
print("There are ", geo_s_WD.shape, "weekday records and ", geo_s_WE.shape, "weekend records.")

#Weekdays: 

lat_long_listD = []
for i in range(1,25):
    temp=[]
    for index, instance in geo_s_WD[geo_s_WD['hour'] == i].iterrows():
        temp.append([instance['Y'],instance['X']])
    lat_long_listD.append(temp)


There are  (28823, 12) weekday records and  (19469, 12) weekend records.


#### Weekdays: 





In [9]:

from branca.element import CssLink, Element, Figure, JavascriptLink
from folium.plugins import HeatMapWithTime

lat_long_listD = []
for i in range(1,25):
    temp=[]
    for index, instance in geo_s_WD[geo_s_WD['hour'] == i].iterrows():
        temp.append([instance['Y'],instance['X']])
    lat_long_listD.append(temp)

mapD = folium.Map(location=[47.60621, -122.33207], zoom_start=12)   #Base map

fig_D=Figure(width=850,height=850)
fig_D.add_child(mapD)

fig_D.header.add_child(
    JavascriptLink('https://rawcdn.githack.com/nezasa/iso8601-js-period/master/iso8601.min.js'),
    name='iso8601')

plugins.HeatMapWithTime(lat_long_listD,radius=8,auto_play=True,position='bottomright').add_to(mapD)

# Save as html, in case this doesn't render in GitHub, to embed 
outfp = "/Users/Owner/Downloads/mapD.html"
mapD.save(outfp)

mapD


#### Weekends:

In [10]:
lat_long_listE = []
for i in range(1,25):
    temp=[]
    for index, instance in geo_s_WE[geo_s_WE['hour'] == i].iterrows():
        temp.append([instance['Y'],instance['X']])
    lat_long_listE.append(temp)

mapE = folium.Map(location=[47.60621, -122.33207], zoom_start=12)   

fig_E=Figure(width=850,height=850)
fig_E.add_child(mapE)

fig_E.header.add_child(
    JavascriptLink('https://rawcdn.githack.com/nezasa/iso8601-js-period/master/iso8601.min.js'),
    name='iso8601')

plugins.HeatMapWithTime(lat_long_listD,radius=8,auto_play=True,position='bottomright').add_to(mapE)

# Save as html, in case this doesn't render in GitHub, to embed 
outfp = "/Users/Owner/Downloads/mapE.html"
mapE.save(outfp)

mapE

#### Injuries only (weekdays and weekends)

In [11]:
geos_INJ = geo_s[(geo_s.SEVERITYDESC == 'Injury Collision') | (geo_s.SEVERITYDESC == 'Serious Injury Collision') | (geo_s.SEVERITYDESC == 'Fatality Collision')] 

lat_long_list_INJ = []

for i in range(1,25):
    temp=[]
    for index, instance in geos_INJ[geos_INJ['hour'] == i].iterrows():
        temp.append([instance['Y'],instance['X']])
    lat_long_list_INJ.append(temp)

mapINJ = folium.Map(location=[47.60621, -122.33207], zoom_start=12)   

fig_INJ=Figure(width=850,height=850)
fig_INJ.add_child(mapINJ)

fig_INJ.header.add_child(
    JavascriptLink('https://rawcdn.githack.com/nezasa/iso8601-js-period/master/iso8601.min.js'),
    name='iso8601')

plugins.HeatMapWithTime(lat_long_list_INJ,radius=8,auto_play=True,position='bottomleft').add_to(mapINJ)


# Save as html, in case this doesn't render in GitHub
outfp = "/Users/Owner/Downloads/mapINJ.html"
mapINJ.save(outfp)

#display here
mapINJ


##### End of Seattle Collisions HeatMap